## Iports

In [1]:
!pip install backtesting

In [2]:
!pip uninstall -y numpy pandas_ta
!pip install numpy==1.23.5 pandas-ta

Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
Found existing installation: pandas_ta 0.3.14b0
Uninstalling pandas_ta-0.3.14b0:
  Successfully uninstalled pandas_ta-0.3.14b0
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached pandas_ta-0.3.14b0-py3-none-any.whl
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.23.5 whic

In [3]:
!pip install pandas_ta
import pandas_ta as ta
!pip install backtesting
!pip install hyperopt

## Functions

### Get data

In [4]:
import pandas as pd
import numpy as np
# import datetime # Removed to prevent overwriting datetime class with datetime module
import requests
import datetime # This import is not needed here as it is already imported below.
import requests
import zipfile
import io
import csv
import os
from datetime import datetime # This imports the datetime class from the datetime module.
from dateutil.relativedelta import relativedelta
from pytz import timezone


# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months


# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [], 'Open': [], 'High': [], 'Low': [], 'Close': [], 'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            # Use datetime.datetime.fromtimestamp to call the correct method.
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone('UTC')))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    return df

### Hyperopt by month

In [5]:
import pandas as pd
from datetime import datetime
from hyperopt import fmin, tpe, hp, Trials
import backtesting
from dateutil.relativedelta import relativedelta
import traceback


def generate_monthly_intervals(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m")
    end = datetime.strptime(end_date, "%Y-%m")
    intervals = []
    while start <= end:
        next_month = start + relativedelta(months=1)
        intervals.append((start.strftime("%Y-%m"), next_month.strftime("%Y-%m")))
        start = next_month
    return intervals

def full_period_backtest(strategy, data, initial_capital=1_000_000):
    data.index = pd.to_datetime(data.index).tz_localize(None)
    test = backtesting.Backtest(data, strategy, cash=initial_capital, commission=0.0005)
    res = test.run()
    trades = res._trades.copy()
    # print(res)
    return trades, res


def analyze_monthly_results(trades, intervals, initial_capital, equity_curve):
    trades['ExitTime'] = pd.to_datetime(trades['ExitTime']).dt.tz_localize(None)
    trades['Close Time'] = pd.to_datetime(trades['ExitTime'])
    trades.set_index('Close Time', inplace=True)
    trades['Trade Duration'] = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds()

    monthly_results = {}
    total_pnl = 0
    profit_months = 0
    loss_months = 0
    total_trade_duration = 0
    total_trades = len(trades)

    capital = initial_capital  # Начальный капитал
    last_interval = intervals[-1]  # Последний месяц

    for start, end in intervals:
        start_date = datetime.strptime(start, "%Y-%m")
        end_date = datetime.strptime(end, "%Y-%m") + relativedelta(months=1, days=-1, hours=23, minutes=59, seconds=59)

        monthly_trades = trades[(trades.index >= start_date) & (trades.index <= end_date)]

        if monthly_trades.empty:
            monthly_results[start] = (0.0, 0)
            continue

        # PnL за месяц
        monthly_pnl = monthly_trades['PnL'].sum()
        capital += monthly_pnl  # Обновляем капитал только на PnL

        # Если это последний месяц, учитываем открытые позиции
        if (start, end) == last_interval and not equity_curve.empty:
            last_equity = equity_curve['Equity'].iloc[-1]  # Финальная Equity
            open_pnl = last_equity - capital  # Открытые позиции
            capital += open_pnl  # Учитываем их в капитале

        # Итоговый процент профита за месяц
        prev_capital = capital - monthly_pnl  # Капитал до учета текущего месяца
        monthly_pnl_percent = ((capital - prev_capital) / prev_capital) * 100 if prev_capital != 0 else 0

        # Средняя длительность сделки (в часах)
        avg_trade_duration = monthly_trades['Trade Duration'].mean() / 3600 if len(monthly_trades) > 0 else 0

        monthly_results[start] = (round(monthly_pnl_percent, 8), round(avg_trade_duration, 2))

        total_pnl += monthly_pnl
        total_trade_duration += monthly_trades['Trade Duration'].sum()

        if monthly_pnl_percent > 0:
            profit_months += 1
        else:
            loss_months += 1

    avg_trade_duration_total = (total_trade_duration / total_trades) / 3600 if total_trades > 0 else 0
    total_months = profit_months + loss_months
    profit_month_ratio = (profit_months / total_months * 100) if total_months > 0 else 0

    # Итоговый Return [%]
    total_return = ((capital - initial_capital) / initial_capital) * 100

    return total_return, profit_months, loss_months, monthly_results, avg_trade_duration_total, profit_month_ratio



def generate_search_space(params_to_optimize):
    space = {}
    for param, settings in params_to_optimize.items():
        if settings['type'] == 'quniform':
            space[param] = hp.quniform(param, settings['range'][0], settings['range'][1], settings.get('q', 1))
        elif settings['type'] == 'uniform':
            space[param] = hp.uniform(param, settings['range'][0], settings['range'][1])
        else:
            raise ValueError(f"Неизвестный тип параметра: {settings['type']}")
    return space

def optimize_strategy(data, symbol, interval, start_date, end_date, shared_folder_path, max_evals, params_to_optimize, fixed_params, strategy):
    intervals = generate_monthly_intervals(start_date, end_date)
    trials = Trials()
    space = generate_search_space(params_to_optimize)
    results = []

    print(f"Общее количество строк в загруженных данных: {len(data)}")
    print(f"Диапазон дат: {data.index.min()} - {data.index.max()}")
    print(f"Оптимизация для {symbol} ({interval}) с {start_date} по {end_date} началась...\n")

    def objective(params):
        for key, value in fixed_params.items():
            setattr(strategy, key, value)

        for key, value in params.items():
            setattr(strategy, key, int(value) if 'length' in key else value)

        print(f"Параметры: {params}, {fixed_params}")

        try:
            trades, res = full_period_backtest(strategy, data, 1_000_000)

            total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration, profit_month_ratio = analyze_monthly_results(
                trades, intervals, 1_000_000, res._equity_curve
            )

            print(f"Общий профит: {total_pnl:.2f}%, Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}, Процент профитных месяцев: {profit_month_ratio:.2f}%")
            print(f"Результаты по месяцам: {[(month, pnl) for month, (pnl, _) in monthly_results.items()]}\n")

            row = {
                'symbol': symbol,
                'interval': interval,
                **{key: fixed_params.get(key, None) for key in fixed_params},
                **{key: (int(value) if 'length' in key else value) for key, value in params.items()},
                'strategy': strategy.__name__,
                'total_pnl%': total_pnl,
                'avg_trade_duration_hours': avg_trade_duration,
                'profit_month_count': profit_months,
                'loss_month_count': loss_months,
                'profit_month_ratio%': profit_month_ratio
            }

            for month, (pnl, _) in monthly_results.items():
                row[month] = pnl

            results.append(row)
            return {'loss': -total_pnl, 'status': 'ok', 'params': params}

        except Exception as e:
            print("Ошибка в objective:", e)
            print(traceback.format_exc())  # Логирование полной трассировки ошибки
            return {'loss': float('inf'), 'status': 'fail', 'params': params}

    try:
        fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    except Exception as e:
        print("Ошибка при выполнении fmin:", e)
        print(traceback.format_exc())
        return None

    results_df = pd.DataFrame(results)
    results_df.to_csv('hyperopt_optimization_results.csv', index=False)
    # save_results_to_csv(results_df, shared_folder_path, "hyperopt_optimization_results.csv") save to the google drive

    if trials.trials:
        best_trial = min(trials.trials, key=lambda x: x['result']['loss'])
        best_params = best_trial['result']['params']
        best_loss = best_trial['result']['loss']
        return best_params, -best_loss
    else:
        print("Не удалось найти лучшие параметры.")
        return None


# best_params, total_pnl = optimize_strategy(
#     data=df,
#     symbol=symbol,
#     interval=interval,
#     start_date=start_date,
#     end_date=end_date,
#     max_evals=max_evals,
#     shared_folder_path=shared_folder_path,
#     params_to_optimize=params_to_optimize,
#     fixed_params=fixed_params
# )

# # Настройка стратегии с оптимизированными параметрами
# SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
# SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
# SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
# SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
# SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

# print("\nНастройка стратегии завершена.")

# # Тестируем стратегию
# test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
# res = test.run()

# print(symbol)
# print()
# print("Результаты с оптимизированными параметрами:", res)
# print('Window length', SlopingStrategy.window_length)
# print('Min space', SlopingStrategy.min_space)
# print('Sloping atr length', SlopingStrategy.sloping_atr_length)
# print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
# print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


## Strategy

### Basic

In [7]:
class UTBot_Strategy(backtesting.Strategy):
    key_value=8
    atr_period=10

    def init(self):
        self.atr = self.I(ta.atr, self.data.High.s, self.data.Low.s, self.data.Close.s, self.atr_period)
        self.x_atr_trailing_stop = [0]
        self.pos = [0]
        self.ema = [0]

    def next(self):
        close = self.data.Close[-1]
        n_loss = self.atr[-1] * self.key_value
        iff_1 = close - n_loss if close > self.x_atr_trailing_stop[-1] else close + n_loss
        iff_2 = (min(self.x_atr_trailing_stop[-1], close + n_loss)
                 if close < self.x_atr_trailing_stop[-1] and self.data.Close[-2] < self.x_atr_trailing_stop[-1] else iff_1)
        x_atr_trailing_stop = (max(self.x_atr_trailing_stop[-1], close - n_loss)
                               if close > self.x_atr_trailing_stop[-1] and self.data.Close[-2] > self.x_atr_trailing_stop[-1]
                               else iff_2)
        iff_3 = (-1 if self.data.Close[-2] > self.x_atr_trailing_stop[-1] and close < self.x_atr_trailing_stop[-1] else
                 self.pos[-1])
        pos = 1 if self.data.Close[-2] < self.x_atr_trailing_stop[-1] and close > self.x_atr_trailing_stop[-1] else iff_3
        ema = close
        above = ema > x_atr_trailing_stop and self.ema[-1] <= self.x_atr_trailing_stop[-1]
        below = x_atr_trailing_stop > ema and self.x_atr_trailing_stop[-1] <= self.ema[-1]
        # определяем сигналы
        buy = close > x_atr_trailing_stop and above
        sell = close < x_atr_trailing_stop and below
        self.x_atr_trailing_stop.append(x_atr_trailing_stop)
        self.pos.append(pos)
        self.ema.append(ema)
        if not self.position and buy:
            self.buy()
        elif self.position and sell:
            self.position.close()

### Basic + SuperTrend

In [7]:
class UTBot_SuperTrend_Strategy(backtesting.Strategy):
    utbot_key_value=8
    utbot_atr_period=14
    supertrend_atr_period=10
    supertrend_multiplier=3.0
    _super_trend = 0

    def init(self):
        self.utbot_atr = self.I(ta.atr, self.data.High.s, self.data.Low.s, self.data.Close.s, self.utbot_atr_period)
        self.supertrend_atr = self.I(ta.atr, self.data.High.s, self.data.Low.s, self.data.Close.s, self.supertrend_atr_period)
        self.x_atr_trailing_stop = [0]
        self.pos = [0]
        self.ema = [0]
        self.up = [0]
        self.down = [0]
        self.trend = [0]

    def next(self):
        close = self.data.Close[-1]
        # SuperTrend
        hl2 = (self.data.High[-1] + self.data.Low[-1]) / 2
        # вычисляем парамер up и up1 (up прошлой свечи)
        self.up.append(hl2 - self.supertrend_multiplier * self.supertrend_atr[-1])
        up1 = self.up[-2] if len(self.up) > 1 else self.up[-1]
        if self.data.Close[-2] > up1:
            self.up[-1] = max(up1, self.up[-1])
        # вычисляем парамер down и down1 (down прошлой свечи)
        self.down.append(hl2 + self.supertrend_multiplier * self.supertrend_atr[-1])
        down1 = self.down[-2] if len(self.down) > 1 else self.down[-1]
        if self.data.Close[-2] < down1:
            self.down[-1] = min(down1, self.down[-1])
        # определяем тренд
        if len(self.trend) == 1:
            # изначально у нас тренд бычий (для первой свечи графика)
            self.trend[-1] = 1
        # если тренд медвежий и цена закрытия больше down1
        if self.trend[-1] == -1 and self.data.Close[-1] > down1:
            # то меняем тренд на бычий
            self.trend.append(1)
        # если тренд бычий и цена закрытия меньше up1
        elif self.trend[-1] == 1 and self.data.Close[-1] < up1:
            # то меняем тренд на медвежий
            self.trend.append(-1)
        else:
            # во других случаях тренд сохраняется
            self.trend.append(self.trend[-1])
        # если тренд изменился с медвежьего на бычий, то формируем сингал на покупку
        buy_signal = self.trend[-2] == -1 and self.trend[-1] == 1
        # если тренд изменился с бычего на медвежий, то формируем сингал на продажу
        sell_signal = self.trend[-2] == 1 and self.trend[-1] == -1
        if buy_signal:
            self._super_trend = 1
        elif sell_signal:
            self._super_trend = -1
        # UT Bot
        n_loss = self.utbot_atr[-1] * self.utbot_key_value
        iff_1 = close - n_loss if close > self.x_atr_trailing_stop[-1] else close + n_loss
        iff_2 = (min(self.x_atr_trailing_stop[-1], close + n_loss)
                 if close < self.x_atr_trailing_stop[-1] and self.data.Close[-2] < self.x_atr_trailing_stop[-1] else iff_1)
        x_atr_trailing_stop = (max(self.x_atr_trailing_stop[-1], close - n_loss)
                               if close > self.x_atr_trailing_stop[-1] and self.data.Close[-2] > self.x_atr_trailing_stop[-1]
                               else iff_2)
        iff_3 = (-1 if self.data.Close[-2] > self.x_atr_trailing_stop[-1] and close < self.x_atr_trailing_stop[-1] else
                 self.pos[-1])
        pos = 1 if self.data.Close[-2] < self.x_atr_trailing_stop[-1] and close > self.x_atr_trailing_stop[-1] else iff_3
        ema = close
        above = ema > x_atr_trailing_stop and self.ema[-1] <= self.x_atr_trailing_stop[-1]
        below = x_atr_trailing_stop > ema and self.x_atr_trailing_stop[-1] <= self.ema[-1]
        buy = close > x_atr_trailing_stop and above
        sell = close < x_atr_trailing_stop and below
        self.x_atr_trailing_stop.append(x_atr_trailing_stop)
        self.pos.append(pos)
        self.ema.append(ema)
        if not self.position and buy and self._super_trend == 1:
            self.buy()
        elif self.position and sell:
            self.position.close()

### UTBot_Strategy_Inner_Trend

In [ ]:
import pandas as pd
import numpy as np
from backtesting import Strategy
import pandas_ta as ta

class UTBot_Strategy_Inner_Trend(Strategy):
    key_value = 8
    atr_period = 10
    rsi_period = 14
    max_trade_duration = 48  # в минутах

    def init(self):
        high, low, close = self.data.High, self.data.Low, self.data.Close

        def calculate_atr():
            df = pd.DataFrame({"high": high, "low": low, "close": close})
            atr = ta.atr(df["high"], df["low"], df["close"], length=int(self.atr_period))
            return atr.to_numpy() if atr is not None else np.zeros(len(close))

        def calculate_rsi():
            df = pd.DataFrame({"close": close})
            rsi = ta.rsi(df["close"], length=int(self.rsi_period))
            return rsi.to_numpy() if rsi is not None else np.zeros(len(close))

        self.atr = self.I(calculate_atr)
        self.rsi = self.I(calculate_rsi)

        self.x_atr_trailing_stop = [0]
        self.pos = [0]
        self.ema = [0]
        self.entry_bar = None

    def next(self):
        close = self.data.Close[-1]
        n_loss = self.atr[-1] * self.key_value

        iff_1 = close - n_loss if close > self.x_atr_trailing_stop[-1] else close + n_loss
        iff_2 = (min(self.x_atr_trailing_stop[-1], close + n_loss)
                 if close < self.x_atr_trailing_stop[-1] and self.data.Close[-2] < self.x_atr_trailing_stop[-1] else iff_1)
        x_atr_trailing_stop = (max(self.x_atr_trailing_stop[-1], close - n_loss)
                               if close > self.x_atr_trailing_stop[-1] and self.data.Close[-2] > self.x_atr_trailing_stop[-1]
                               else iff_2)

        iff_3 = (-1 if self.data.Close[-2] > self.x_atr_trailing_stop[-1] and close < self.x_atr_trailing_stop[-1] else
                 self.pos[-1])
        pos = 1 if self.data.Close[-2] < self.x_atr_trailing_stop[-1] and close > self.x_atr_trailing_stop[-1] else iff_3

        ema = close
        above = ema > x_atr_trailing_stop and self.ema[-1] <= self.x_atr_trailing_stop[-1]
        below = x_atr_trailing_stop > ema and self.x_atr_trailing_stop[-1] <= self.ema[-1]

        buy = (close > x_atr_trailing_stop and above) or (self.position and close > self.x_atr_trailing_stop[-1])
        sell = close < x_atr_trailing_stop and below

        if buy and self.rsi[-1] > 50:
            if not self.position:
                self.buy(size=1)
                self.entry_bar = self.data.index[-1]
            elif self.position and close > self.x_atr_trailing_stop[-1]:
                self.buy(size=1)

        if self.position:
            if sell or (self.entry_bar and (self.data.index[-1] - self.entry_bar) >= pd.Timedelta(self.max_trade_duration, unit="min")):
                self.position.close()

        self.x_atr_trailing_stop.append(x_atr_trailing_stop)
        self.pos.append(pos)
        self.ema.append(ema)


Общее количество строк в загруженных данных: 123264
Диапазон дат: 2023-12-01 00:00:00 - 2025-01-31 23:55:00
Оптимизация для BTCUSDT (5m) с 2023-12 по 2025-01 началась...

Параметры: {'atr_period': 5.0, 'key_value': 7.0, 'max_trade_duration': 7.876636535392729, 'rsi_period': 3.0}, {}
  0%|          | 0/70 [00:00<?, ?trial/s, best loss=?]

Backtest.run:   0%|          | 0/123258 [00:00<?, ?it/s]

Общий профит: -13.80%, Профитные месяцы: 9, Убыточные месяцы: 5, Процент профитных месяцев: 64.29%
Результаты по месяцам: [('2023-12', -0.34782), ('2024-01', -0.34576263), ('2024-02', 0.34330699), ('2024-03', 0.18899422), ('2024-04', -0.21894715), ('2024-05', -0.20452068), ('2024-06', -0.19616847), ('2024-07', 0.0072264), ('2024-08', 0.12969292), ('2024-09', 0.07794242), ('2024-10', 0.08360418), ('2024-11', 0.8031576), ('2024-12', 0.81122278), ('2025-01', 0.14922448)]

Параметры: {'atr_period': 12.0, 'key_value': 18.0, 'max_trade_duration': 87.92743876807968, 'rsi_period': 14.0}, {}
  1%|▏         | 1/70 [00:02<03:08,  2.74s/trial, best loss: 13.799889830000081]

Backtest.run:   0%|          | 0/123249 [00:00<?, ?it/s]

Общий профит: -15.56%, Профитные месяцы: 8, Убыточные месяцы: 6, Процент профитных месяцев: 57.14%
Результаты по месяцам: [('2023-12', 0.68237), ('2024-01', -0.05136947), ('2024-02', 1.53317106), ('2024-03', 0.05845939), ('2024-04', 0.79749029), ('2024-05', -0.4532331), ('2024-06', -2.35521094), ('2024-07', 0.04634332), ('2024-08', -0.84123759), ('2024-09', -0.87123874), ('2024-10', 0.83822726), ('2024-11', 4.21442724), ('2024-12', 2.35817168), ('2025-01', -1.26225694)]

Параметры: {'atr_period': 14.0, 'key_value': 11.0, 'max_trade_duration': 10.482994823908198, 'rsi_period': 15.0}, {}
  3%|▎         | 2/70 [00:06<03:38,  3.21s/trial, best loss: 13.799889830000081]

...

Backtest.run:   0%|          | 0/123246 [00:00<?, ?it/s]

Общий профит: -16.72%, Профитные месяцы: 8, Убыточные месяцы: 6, Процент профитных месяцев: 57.14%
Результаты по месяцам: [('2023-12', -0.60342), ('2024-01', -0.13640308), ('2024-02', 1.99135612), ('2024-03', 2.03924153), ('2024-04', 0.66102225), ('2024-05', -1.70030342), ('2024-06', -0.82547985), ('2024-07', 2.18018325), ('2024-08', 2.35982169), ('2024-09', 0.57452006), ('2024-10', 1.26972139), ('2024-11', 0.311647), ('2024-12', -3.25328827), ('2025-01', -2.79182117)]

Параметры: {'atr_period': 11.0, 'key_value': 1.0, 'max_trade_duration': 65.66987599027635, 'rsi_period': 11.0}, {}
 87%|████████▋ | 61/70 [04:27<00:39,  4.33s/trial, best loss: -25.308930000000263]

Backtest.run:   0%|          | 0/123252 [00:00<?, ?it/s]

Общий профит: 24.69%, Профитные месяцы: 7, Убыточные месяцы: 4, Процент профитных месяцев: 63.64%
Результаты по месяцам: [('2023-12', 1.70927), ('2024-01', 7.25292788), ('2024-02', 9.61796532), ('2024-03', 3.18875439), ('2024-04', 2.4120137), ('2024-05', 0.53133208), ('2024-06', -0.33713341), ('2024-07', -0.40506928), ('2024-08', -1.09289964), ('2024-09', 0.10505979), ('2024-10', -0.12636715), ('2024-11', 0.0), ('2024-12', 0.0), ('2025-01', 0.0)]

Параметры: {'atr_period': 7.0, 'key_value': 2.0, 'max_trade_duration': 68.70301494713107, 'rsi_period': 14.0}, {}
 89%|████████▊ | 62/70 [04:33<00:39,  5.00s/trial, best loss: -25.308930000000263]

Backtest.run:   0%|          | 0/123249 [00:00<?, ?it/s]

## Backtest

### UTBot_Strategy

In [8]:
symbol_1 = 'BTCUSDT'
interval_1 = '5m'
start_date_1 = '2023-12'
end_date_1 = '2025-01'
max_evals_1 = 70
strategy = UTBot_Strategy

# Параметры для оптимизации
params_to_optimize_1 = {
    'key_value': {'type': 'quniform', 'range': (1, 20, 1)},
    'atr_period': {'type': 'quniform', 'range': (1, 20, 1)}

}

fixed_params_1 = {}
# # Фиксированные параметры
# fixed_params_1 = {
#     'min_space': 5,
#     # 'window_length': 50,
#     # 'sloping_atr_length': 20,
#     # 'take_profit_multiplier': 1.5,
#     # 'stop_loss_multiplier': 0.5
# }
fixed_params_1 = {}

In [9]:
# Настройка и вызов оптимизации
# symbol = 'SUIUSDT'
# interval = '1h'
# start_date = '2024-11'
# end_date = '2024-11'

symbol = symbol_1
interval = interval_1
start_date = start_date_1
end_date = end_date_1
max_evals = max_evals_1
params_to_optimize=params_to_optimize_1
fixed_params=fixed_params_1

# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)
print(symbol, interval)
print(f"Общее количество строк в загруженных данных: {len(df)}")
print(f"Диапазон дат: {df.index.min()} - {df.index.max()}")

BTCUSDT 5m
Общее количество строк в загруженных данных: 123264
Диапазон дат: 2023-12-01 00:00:00+00:00 - 2025-01-31 23:55:00+00:00


In [10]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=None,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params,
    strategy = strategy
)

# 🔹 Автоматическое обновление параметров стратегии
for param_name, param_value in {**fixed_params, **best_params}.items():
    if hasattr(strategy, param_name):
        setattr(strategy, param_name, int(param_value) if 'length' in param_name else param_value)

print("\n✅ Настройка стратегии завершена.")

# 🔹 Запуск бэктеста с оптимизированными параметрами
test = backtesting.Backtest(data=df, strategy=strategy, cash=1_000_000, commission=0.0005)
res = test.run()

# 🔹 Выводим результаты
print(f"\n🔹 {symbol} {interval}")
print("📊 Результаты с оптимизированными параметрами:")
print(res)

# 🔹 Выводим актуальные параметры стратегии
for param_name in dir(strategy):
    if not param_name.startswith("__") and not callable(getattr(strategy, param_name)):
        print(f"{param_name}: {getattr(strategy, param_name)}")

Общее количество строк в загруженных данных: 123264
Диапазон дат: 2023-12-01 00:00:00+00:00 - 2025-01-31 23:55:00+00:00
Оптимизация для BTCUSDT (5m) с 2023-12 по 2025-01 началась...

Параметры: {'atr_period': 5.0, 'key_value': 18.0}, {}
  0%|          | 0/70 [00:00<?, ?trial/s, best loss=?]

Backtest.run:   0%|          | 0/123258 [00:00<?, ?it/s]

Общий профит: 39.58%, Профитные месяцы: 9, Убыточные месяцы: 5, Процент профитных месяцев: 64.29%
Результаты по месяцам: [('2023-12', 8.89445), ('2024-01', 4.2459556), ('2024-02', 31.25948417), ('2024-03', 10.39133248), ('2024-04', -0.66442478), ('2024-05', 0.71734078), ('2024-06', -2.63044262), ('2024-07', 2.25874756), ('2024-08', -3.00458853), ('2024-09', 10.28157936), ('2024-10', 19.71632874), ('2024-11', 8.46127044), ('2024-12', -5.81581391), ('2025-01', -8.38187034)]

Параметры: {'atr_period': 16.0, 'key_value': 6.0}, {}
  1%|▏         | 1/70 [00:03<04:03,  3.53s/trial, best loss: -39.58470790000011]

Backtest.run:   0%|          | 0/123247 [00:00<?, ?it/s]

  1%|▏         | 1/70 [00:05<05:57,  5.18s/trial, best loss: -39.58470790000011]


KeyboardInterrupt: 

### UTBot_Strategy_Supertrend

In [11]:
# Параметры для оптимизации
strategy_2 = UTBot_SuperTrend_Strategy
max_evals_1 = 70

params_to_optimize_2 = {
    'utbot_key_value': {'type': 'quniform', 'range': (3, 20, 1)},
    'utbot_atr_period': {'type': 'quniform', 'range': (3, 20, 1)},
    'supertrend_atr_period': {'type': 'quniform', 'range': (3, 20, 1)},
    'supertrend_multiplier': {'type': 'uniform', 'range': (1.0, 5.5)},

}

fixed_params_2 = {}
# # Фиксированные параметры
# fixed_params_2 = {
#     'min_space': 5,
#     # 'window_length': 50,
#     # 'sloping_atr_length': 20,
#     # 'take_profit_multiplier': 1.5,
#     # 'stop_loss_multiplier': 0.5
# }


params_to_optimize=params_to_optimize_2
fixed_params=fixed_params_2

In [12]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=None,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params,
    strategy = strategy_2
)

# 🔹 Автоматическое обновление параметров стратегии
for param_name, param_value in {**fixed_params, **best_params}.items():
    if hasattr(strategy, param_name):
        setattr(strategy, param_name, int(param_value) if 'length' in param_name else param_value)

print("\n✅ Настройка стратегии завершена.")

# 🔹 Запуск бэктеста с оптимизированными параметрами
test = backtesting.Backtest(data=df, strategy=strategy, cash=1_000_000, commission=0.0005)
res = test.run()

# 🔹 Выводим результаты
print(f"\n🔹 {symbol} {interval}")
print("📊 Результаты с оптимизированными параметрами:")
print(res)

# 🔹 Выводим актуальные параметры стратегии
for param_name in dir(strategy):
    if not param_name.startswith("__") and not callable(getattr(strategy, param_name)):
        print(f"{param_name}: {getattr(strategy, param_name)}")

Общее количество строк в загруженных данных: 123264
Диапазон дат: 2023-12-01 00:00:00 - 2025-01-31 23:55:00
Оптимизация для BTCUSDT (5m) с 2023-12 по 2025-01 началась...

Параметры: {'supertrend_atr_period': 9.0, 'supertrend_multiplier': 3.652296306705788, 'utbot_atr_period': 20.0, 'utbot_key_value': 15.0}, {}
  0%|          | 0/70 [00:00<?, ?trial/s, best loss=?]

Backtest.run:   0%|          | 0/123243 [00:00<?, ?it/s]

Общий профит: 5.59%, Профитные месяцы: 6, Убыточные месяцы: 8, Процент профитных месяцев: 42.86%
Результаты по месяцам: [('2023-12', -4.40437), ('2024-01', 1.13549124), ('2024-02', 23.65184885), ('2024-03', 6.6344492), ('2024-04', -3.130029), ('2024-05', -1.26147074), ('2024-06', -5.10775689), ('2024-07', -2.67172673), ('2024-08', -6.90591434), ('2024-09', 11.32850614), ('2024-10', 22.65795653), ('2024-11', 9.82628809), ('2024-12', -6.03177997), ('2025-01', -7.98691682)]

Параметры: {'supertrend_atr_period': 16.0, 'supertrend_multiplier': 4.555277703469134, 'utbot_atr_period': 10.0, 'utbot_key_value': 10.0}, {}
  1%|▏         | 1/70 [00:03<03:37,  3.16s/trial, best loss: -5.588272685000207]

Backtest.run:   0%|          | 0/123247 [00:00<?, ?it/s]

Общий профит: 15.03%, Профитные месяцы: 8, Убыточные месяцы: 6, Процент профитных месяцев: 57.14%
Результаты по месяцам: [('2023-12', -3.38261), ('2024-01', 26.79394465), ('2024-02', 25.79987756), ('2024-03', -1.72350941), ('2024-04', -0.3353801), ('2024-05', 2.96396611), ('2024-06', 4.67037003), ('2024-07', 3.9999673), ('2024-08', -4.23609454), ('2024-09', -1.68824784), ('2024-10', 11.98835419), ('2024-11', 5.51094832), ('2024-12', -3.6572552), ('2025-01', 0.80343306)]

Параметры: {'supertrend_atr_period': 9.0, 'supertrend_multiplier': 3.377247262044064, 'utbot_atr_period': 9.0, 'utbot_key_value': 7.0}, {}
  3%|▎         | 2/70 [00:07<04:06,  3.62s/trial, best loss: -15.030879094999936]

...

Общий профит: 30.09%, Профитные месяцы: 10, Убыточные месяцы: 4, Процент профитных месяцев: 71.43%
Результаты по месяцам: [('2023-12', -2.24188), ('2024-01', 26.0347478), ('2024-02', 17.99780374), ('2024-03', -4.86280657), ('2024-04', 2.22835077), ('2024-05', 2.14706118), ('2024-06', 2.31085269), ('2024-07', 7.04900764), ('2024-08', 5.0288982), ('2024-09', 11.44110585), ('2024-10', 16.78840774), ('2024-11', 3.11452232), ('2024-12', -7.29267933), ('2025-01', -2.88354142)]

Параметры: {'supertrend_atr_period': 17.0, 'supertrend_multiplier': 1.941108367589396, 'utbot_atr_period': 11.0, 'utbot_key_value': 15.0}, {}
 99%|█████████▊| 69/70 [04:07<00:03,  3.75s/trial, best loss: -58.91653661000002]

Backtest.run:   0%|          | 0/123246 [00:00<?, ?it/s]

Общий профит: 32.67%, Профитные месяцы: 9, Убыточные месяцы: 5, Процент профитных месяцев: 64.29%
Результаты по месяцам: [('2023-12', -2.24219), ('2024-01', 4.93257777), ('2024-02', 28.02428237), ('2024-03', 10.2619689), ('2024-04', 1.00246692), ('2024-05', 2.39418869), ('2024-06', -2.34450445), ('2024-07', 2.17886361), ('2024-08', -1.61450196), ('2024-09', 10.96638842), ('2024-10', 20.13808285), ('2024-11', 10.15448036), ('2024-12', -3.79228959), ('2025-01', -7.3194528)]

100%|██████████| 70/70 [04:10<00:00,  3.59s/trial, best loss: -58.91653661000002]

✅ Настройка стратегии завершена.


Backtest.run:   0%|          | 0/123258 [00:00<?, ?it/s]


🔹 BTCUSDT 5m
📊 Результаты с оптимизированными параметрами:
Start                     2023-12-01 00:00:00
End                       2025-01-31 23:55:00
Duration                    427 days 23:55:00
Exposure Time [%]                    53.85271
Equity Final [$]                1904579.78355
Equity Peak [$]                 2251465.26435
Commissions [$]                  205514.31645
Return [%]                           90.45798
Buy & Hold Return [%]                172.1424
Return (Ann.) [%]                    69.45189
Volatility (Ann.) [%]                64.82673
CAGR [%]                             73.22705
Sharpe Ratio                          1.07135
Sortino Ratio                         3.66874
Calmar Ratio                          4.37699
Alpha [%]                            15.34418
Beta                                  0.43635
Max. Drawdown [%]                   -15.86749
Avg. Drawdown [%]                    -1.08418
Max. Drawdown Duration      131 days 20:00:00
Avg. Drawdown Durati

### UTBot_Strategy_Inner_Trend

In [17]:
# Параметры для оптимизации
strategy_3 = UTBot_Strategy_Inner_Trend
max_evals_1 = 70

params_to_optimize_3 = {
    'key_value': {'type': 'quniform', 'range': (1, 20, 1)},
    'atr_period': {'type': 'quniform', 'range': (1, 20, 1)},
    'rsi_period': {'type': 'quniform', 'range': (1, 20, 1)},
    'max_trade_duration': {'type': 'uniform', 'range': (2, 100, 2)},

}


fixed_params_3 = {}
# # Фиксированные параметры
# fixed_params_2 = {
#     'min_space': 5,
#     # 'window_length': 50,
#     # 'sloping_atr_length': 20,
#     # 'take_profit_multiplier': 1.5,
#     # 'stop_loss_multiplier': 0.5
# }


params_to_optimize=params_to_optimize_3
fixed_params=fixed_params_3

In [25]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=None,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params,
    strategy = strategy_3
)

# 🔹 Автоматическое обновление параметров стратегии
for param_name, param_value in {**fixed_params, **best_params}.items():
    if hasattr(strategy, param_name):
        setattr(strategy, param_name, int(param_value) if 'length' in param_name else param_value)

print("\n✅ Настройка стратегии завершена.")

# 🔹 Запуск бэктеста с оптимизированными параметрами
test = backtesting.Backtest(data=df, strategy=strategy, cash=1_000_000, commission=0.0005)
res = test.run()

# 🔹 Выводим результаты
print(f"\n🔹 {symbol} {interval}")
print("📊 Результаты с оптимизированными параметрами:")
print(res)

# 🔹 Выводим актуальные параметры стратегии
for param_name in dir(strategy):
    if not param_name.startswith("__") and not callable(getattr(strategy, param_name)):
        print(f"{param_name}: {getattr(strategy, param_name)}")

Общее количество строк в загруженных данных: 123264
Диапазон дат: 2023-12-01 00:00:00 - 2025-01-31 23:55:00
Оптимизация для BTCUSDT (5m) с 2023-12 по 2025-01 началась...

Параметры: {'atr_period': 10.0, 'key_value': 16.0, 'max_trade_duration': 78.65016791562738, 'rsi_period': 1.0}, {}
Ошибка в objective:
module 'pandas_ta' has no attribute 'ATR'
Traceback (most recent call last):
  File "<ipython-input-5-da69c00ffda7>", line 123, in objective
    trades, res = full_period_backtest(strategy, data, 1_000_000)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-5-da69c00ffda7>", line 22, in full_period_backtest
    res = test.run()
          ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/backtesting/backtesting.py", line 1298, in run
    strategy.init()
  File "<ipython-input-16-2b791077df70>", line 11, in init
    self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_period)
                      ^^^^^^
Attribu

TypeError: cannot unpack non-iterable NoneType object